In [67]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [7]:
""" selected_pairs = pd.read_csv("C:/Users/19712/Desktop/Berkeley MFE/Semester 4/IAQF/SelectedPairs.csv")
selected_pairs = selected_pairs['Ticker Pair']
selected_pairs = pd.DataFrame(selected_pairs)
selected_pairs """

,Ticker Pair
0,ISCB_IYH
1,ISCB_IXJ
2,ISCB_IYK
3,ISCB_IYC
4,ISCG_IYC
5,ISCB_IYR
6,IWC_IYR
7,IWC_IYH
8,EIS_TUR
9,IWC_IXJ


In [8]:
selected_pairs = pd.DataFrame(['ISCB_IYH','ISCB_IXJ','ISCB_IYK','ISCB_IYC','ISCG_IYC','ISCB_IYR','IWC_IYR','IWC_IYH','EIS_TUR'])
selected_pairs.columns = ['Ticker Pair']

In [9]:
#features = pd.read_csv("C:/Users/19712/Desktop/Berkeley MFE/Semester 4/IAQF/ETF_Screen_Information_2023_02_09.csv")
features = pd.read_csv('ETF_Screen_Information.csv', index_col=0)


In [10]:
features.columns

Index(['Date', 'ETF_Ticker', 'Average Spread', 'AUM', 'Count', 'Div Yield',
       'Price to Book', 'Price to Earnings', 'Total Return', 'Dollar Volume',
       'ETF Price', 'ETF_Len'],
      dtype='object')

In [11]:
features.head()

,Date,ETF_Ticker,Average Spread,AUM,Count,Div Yield,Price to Book,Price to Earnings,Total Return,Dollar Volume,ETF Price,ETF_Len
416,2008-01-02,*XCS,NaN,2.792085e+07,217.0,0.039010,1.761031,59.699048,18.435817,NaN,17.467957,4
417,2008-01-02,*XEG,NaN,5.492800e+08,61.0,0.024876,2.924148,20.516180,26.531370,NaN,22.252901,4
420,2008-01-02,*XFN,NaN,5.332795e+08,29.0,0.028454,2.183175,13.263941,30.330787,NaN,26.252596,4
421,2008-01-02,*XGD,NaN,1.125968e+09,37.0,0.005080,2.517170,144.230461,20.947008,NaN,19.519732,4
422,2008-01-02,*XIC,NaN,4.839745e+08,259.0,0.021631,2.586234,19.966194,27.812710,NaN,22.083185,4


In [12]:
search_tickers = []
for pair in selected_pairs['Ticker Pair'].to_list():
    pair_split = pair.split('_')
    search_tickers.append(pair_split[0])
    search_tickers.append(pair_split[1])
    
search_tickers = list(set(search_tickers))

price_df = features[['Date', 'ETF_Ticker', 'ETF Price']]
price_df = price_df.set_index(['ETF_Ticker'])
price_df = price_df.loc[search_tickers]
#price_df = price_df.reset_index().pivot(index = 'Date', columns = 'ETF_Ticker')
#price_df.to_csv('C:/Users/19712/Desktop/Berkeley MFE/Semester 4/IAQF/price_df.csv')


In [68]:

def feature_generator_ratio(ticker1, ticker2, features):
    iter_df = features.set_index(['ETF_Ticker'])
    iter_df = iter_df.loc[[ticker1, ticker2]]
    iter_df = iter_df[['Date', 'Div Yield', 'Price to Book', 'Price to Earnings']]
    iter_df['Date'] = pd.to_datetime(iter_df.Date)
    iter_df = iter_df.sort_values('Date')
    
    #Divide the dfs
    iter_1  = iter_df.loc[ticker1].reset_index().set_index(['Date']).drop(['ETF_Ticker'],axis=1)
    iter_2 = iter_df.loc[ticker2].reset_index().set_index(['Date']).drop(['ETF_Ticker'],axis=1)
    iter_df = iter_1.divide(iter_2)
    
    return iter_df

def period_returns(ticker1, ticker2, features):
    iter_df = features.set_index(['ETF_Ticker'])
    iter_df = iter_df.loc[[ticker1, ticker2]]
    iter_df = iter_df[['Date', 'Total Return']]
    iter_df['Date'] = pd.to_datetime(iter_df.Date)
    iter_df = iter_df.sort_values('Date')
    
    iter_1_ret = iter_df.loc[ticker1].reset_index().set_index(['Date']).drop(['ETF_Ticker'],axis=1)
    iter_2_ret = iter_df.loc[ticker2].reset_index().set_index(['Date']).drop(['ETF_Ticker'],axis=1)
    
    #Calculate daily trailing returns
    iter_1_ret['Total_Ret_Pct_1'] = iter_1_ret['Total Return'].pct_change(periods=1)
    iter_2_ret['Total_Ret_Pct_1'] = iter_2_ret['Total Return'].pct_change(periods=1)
    iter_1_ret['Total_Ret_Pct_5'] = iter_1_ret['Total Return'].pct_change(periods=5)
    iter_2_ret['Total_Ret_Pct_5'] = iter_2_ret['Total Return'].pct_change(periods=5)
    iter_1_ret['Total_Ret_Pct_21'] = iter_1_ret['Total Return'].pct_change(periods=21)
    iter_2_ret['Total_Ret_Pct_21'] = iter_2_ret['Total Return'].pct_change(periods=21)
    iter_1_ret['Total_Ret_Pct_63'] = iter_1_ret['Total Return'].pct_change(periods=63)
    iter_2_ret['Total_Ret_Pct_63'] = iter_2_ret['Total Return'].pct_change(periods=63)
    iter_1_ret['Total_Ret_Pct_252'] = iter_1_ret['Total Return'].pct_change(periods=252)
    iter_2_ret['Total_Ret_Pct_252'] = iter_2_ret['Total Return'].pct_change(periods=252)
    
    #Calculate fwd daily trailing returns
    iter_1_ret['Y_Fwd_Total_Ret_Pct_1'] = iter_1_ret['Total Return'].pct_change(periods=-1)
    iter_2_ret['Y_Fwd_Total_Ret_Pct_1'] = iter_2_ret['Total Return'].pct_change(periods=-1)
    iter_1_ret['Y_Fwd_Total_Ret_Pct_5'] = iter_1_ret['Total Return'].pct_change(periods=-5)
    iter_2_ret['Y_Fwd_Total_Ret_Pct_5'] = iter_2_ret['Total Return'].pct_change(periods=-5)
    
    iter_df = iter_1_ret.subtract(iter_2_ret).round(4).drop(['Total Return'], axis=1)
    
    return iter_df

def cal_features(index1, index2, features):

    df = features.copy()
    df.Date = pd.to_datetime(df.Date)
    df.index = df.Date
    df.drop('Date', axis=1, inplace=True)

    p1 = df[df['ETF_Ticker']==index1]['Total Return']
    p2 = df[df['ETF_Ticker']==index2]['Total Return']

    price_df = pd.concat([p1,p2],axis=1).dropna().sort_index()
    price_df.columns = [['Total Return 1', 'Total Return 2']]

    y = price_df['Total Return 1']
    X = price_df.drop('Total Return 1',axis=1)

    X = sm.add_constant(X)
    result = sm.OLS(y, X).fit()

    days_s_cross = result.resid.sort_index().rolling(2).apply(lambda x: 0 if np.sign(x[1]) != np.sign(x[0]) else 1).fillna(0)
    no_reset_arr = days_s_cross.cumsum()
    reset_arr = days_s_cross == 0
    excess = np.maximum.accumulate(reset_arr * no_reset_arr)
    days_s_cross = no_reset_arr - excess 

    n_zero_cross = result.resid.sort_index().rolling(2).apply(lambda x: 1 if np.sign(x[1]) != np.sign(x[0]) else 0)
    n_zero_cross = n_zero_cross.fillna(0).cumsum()
    price_df['zero_cross'] = n_zero_cross
    price_df['days_s_cross'] = days_s_cross

    price_df.columns = price_df.columns.levels[0]
    return price_df

def build_feature_df(ticker_list, features):
    container_df = pd.DataFrame()
    
    for ticker_pair in tqdm(ticker_list):
        ticker_split = ticker_pair.split('_')
        val_df = feature_generator_ratio(ticker_split[0], ticker_split[1], features)
        ret_df = period_returns(ticker_split[0], ticker_split[1], features)
        cross_df = cal_features(ticker_split[0], ticker_split[1], features)
        feature_df = val_df.join(ret_df)
        feature_df = pd.concat([feature_df,cross_df],axis=1).reset_index()
        feature_df['Ticker_Pair'] = ticker_split[0] +'_'+ticker_split[1]
        container_df = pd.concat([container_df, feature_df])
        
    return container_df

final_training_df = build_feature_df(selected_pairs['Ticker Pair'].to_list(), features)

100%|██████████| 9/9 [00:10<00:00,  1.17s/it]


In [70]:
final_training_df.describe()
#import matplotlib.pyplot as plt
#for col in final_training_df.columns:
#    plt.title(col)
#    plt.hist(final_training_df[col], bins = 50)
#    plt.show()

,Div Yield,Price to Book,Price to Earnings,Total_Ret_Pct_1,Total_Ret_Pct_5,Total_Ret_Pct_21,Total_Ret_Pct_63,Total_Ret_Pct_252,Y_Fwd_Total_Ret_Pct_1,Y_Fwd_Total_Ret_Pct_5,Total Return 1,Total Return 2,zero_cross,days_s_cross
count,38165.000000,38165.000000,38165.000000,38156.000000,38120.000000,37976.000000,37598.000000,35897.000000,38156.000000,38120.000000,38165.000000,38165.000000,38165.000000,38165.000000
mean,0.642160,0.734482,1.034124,-0.000035,-0.000153,-0.000841,-0.003009,-0.010831,0.000055,0.000290,48.050658,105.671375,63.080126,140.827198
std,0.355053,0.294369,105.005573,0.011838,0.025011,0.049311,0.086108,0.170256,0.011883,0.025488,30.435562,81.608500,40.788745,152.381778
min,0.143757,0.270713,-8380.372502,-0.224500,-0.274300,-0.461800,-0.699100,-1.416600,-0.221400,-0.341500,6.178609,8.670185,0.000000,0.000000
25%,0.368576,0.524142,0.667079,-0.005600,-0.012500,-0.027500,-0.051400,-0.111800,-0.005300,-0.011900,23.930362,46.821993,32.000000,24.000000
50%,0.559560,0.663693,1.183775,-0.000050,-0.000200,-0.000800,-0.007000,-0.025000,0.000000,0.000200,43.484480,75.643086,60.000000,87.000000
75%,0.828240,0.879692,1.827151,0.005400,0.012100,0.025300,0.041900,0.072600,0.005600,0.012500,59.306118,153.198079,94.000000,208.000000
max,2.565965,2.636377,13944.342325,0.185400,0.253500,0.353700,0.501700,1.307300,0.190300,0.222300,187.993513,392.499697,156.000000,836.000000


In [80]:
final_training_df.to_csv('C:/Users/19712/Desktop/Berkeley MFE/Semester 4/IAQF/Example_Training_df.csv')